In [2]:
import pandas as pd

In [51]:
train_df = pd.read_csv('../data/train.csv', encoding = 'utf-8-sig')
test_df = pd.read_csv('../data/test.csv', encoding = 'utf-8-sig')

# Cleaning

In [52]:
# 데이터 형식 검증 및 에러 데이터 분리
print("=== Train Dataset Format Check ===")

error_train_df = pd.DataFrame()

# 공사종류 형식 검증 ('건축 / 건축물 / 근린생활시설' 형식)
print("\n공사종류 형식 검증:")
invalid_format = train_df[~train_df['공사종류'].str.contains(' / ', na=False)]
if len(invalid_format) > 0:
    print(f"잘못된 형식 발견: {len(invalid_format)}건")
    error_train_df = pd.concat([error_train_df, invalid_format])
    train_df = train_df[train_df['공사종류'].str.contains(' / ', na=False)]
else:
    print("모든 데이터가 올바른 형식입니다.")

# 공종 형식 검증 ('건축 > 가설공사' 형식) 
print("\n공종 형식 검증:")
invalid_format = train_df[~train_df['공종'].str.contains(' > ', na=False)]
if len(invalid_format) > 0:
    print(f"잘못된 형식 발견: {len(invalid_format)}건")
    error_train_df = pd.concat([error_train_df, invalid_format])
    train_df = train_df[train_df['공종'].str.contains(' > ', na=False)]
else:
    print("모든 데이터가 올바른 형식입니다.")

# 사고객체 형식 검증 ('시설물 > 가설계단' 형식)
print("\n사고객체 형식 검증:")
invalid_format = train_df[~train_df['사고객체'].str.contains(' > ', na=False)]
if len(invalid_format) > 0:
    print(f"잘못된 형식 발견: {len(invalid_format)}건")
    error_train_df = pd.concat([error_train_df, invalid_format])
    train_df = train_df[train_df['사고객체'].str.contains(' > ', na=False)]
else:
    print("모든 데이터가 올바른 형식입니다.")

# 작업프로세스, 사고원인 NaN값 검증
print("\n작업프로세스, 사고원인 NaN값 검증:")
invalid_format = train_df[train_df['작업프로세스'].isna() | train_df['사고원인'].isna()]
if len(invalid_format) > 0:
    print(f"NaN값 발견: {len(invalid_format)}건")
    error_train_df = pd.concat([error_train_df, invalid_format])
    train_df = train_df[~(train_df['작업프로세스'].isna() | train_df['사고원인'].isna())]
else:
    print("NaN값이 없습니다.")

print("\n=== Test Dataset Format Check ===")

# 공사종류 형식 검증
print("\n공사종류 형식 검증:")
invalid_format = test_df[~test_df['공사종류'].str.contains(' / ', na=False)]
if len(invalid_format) > 0:
    print(f"잘못된 형식 발견: {len(invalid_format)}건")
else:
    print("모든 데이터가 올바른 형식입니다.")

# 공종 형식 검증
print("\n공종 형식 검증:")
invalid_format = test_df[~test_df['공종'].str.contains(' > ', na=False)]
if len(invalid_format) > 0:
    print(f"잘못된 형식 발견: {len(invalid_format)}건")
else:
    print("모든 데이터가 올바른 형식입니다.")

# 사고객체 형식 검증
print("\n사고객체 형식 검증:")
invalid_format = test_df[~test_df['사고객체'].str.contains(' > ', na=False)]
if len(invalid_format) > 0:
    print(f"잘못된 형식 발견: {len(invalid_format)}건")
    print(f"{invalid_format['사고객체']}")
else:
    print("모든 데이터가 올바른 형식입니다.")

# 작업프로세스, 사고원인 NaN값 검증 (test_df)
print("\n작업프로세스, 사고원인 NaN값 검증:")
invalid_format = test_df[test_df['작업프로세스'].isna() | test_df['사고원인'].isna()]
if len(invalid_format) > 0:
    print(f"NaN값 발견: {len(invalid_format)}건")
else:
    print("NaN값이 없습니다.")

# 중복 제거
error_train_df = error_train_df.drop_duplicates()
print(f"\n총 {len(error_train_df)}개의 잘못된 형식 데이터가 error_train_df로 분리되었습니다.")

# error train data
error_train_df.to_csv('../data/error_train.csv', index=False)


=== Train Dataset Format Check ===

공사종류 형식 검증:
잘못된 형식 발견: 1건

공종 형식 검증:
잘못된 형식 발견: 10건

사고객체 형식 검증:
잘못된 형식 발견: 684건

작업프로세스, 사고원인 NaN값 검증:
NaN값 발견: 126건

=== Test Dataset Format Check ===

공사종류 형식 검증:
모든 데이터가 올바른 형식입니다.

공종 형식 검증:
모든 데이터가 올바른 형식입니다.

사고객체 형식 검증:
잘못된 형식 발견: 1건
710    NaN
Name: 사고객체, dtype: object

작업프로세스, 사고원인 NaN값 검증:
NaN값이 없습니다.

총 821개의 잘못된 형식 데이터가 error_train_df로 분리되었습니다.


In [53]:
# train_df의 index 재정렬
train_df = train_df.reset_index(drop=True)

# Formatting

In [54]:
# 데이터 전처리
train_df['공사종류(대분류)'] = train_df['공사종류'].str.split(' / ').str[0]
train_df['공사종류(중분류)'] = train_df['공사종류'].str.split(' / ').str[1]
train_df['공종(대분류)'] = train_df['공종'].str.split(' > ').str[0]
train_df['공종(중분류)'] = train_df['공종'].str.split(' > ').str[1]
train_df['사고객체(대분류)'] = train_df['사고객체'].str.split(' > ').str[0]
train_df['사고객체(중분류)'] = train_df['사고객체'].str.split(' > ').str[1]
train_df['인적사고'] = train_df['인적사고'].apply(lambda x: '떨어짐' if isinstance(x, str) and '떨어짐' in x else x)
train_df['인적사고'] = train_df['인적사고'].apply(lambda x: '넘어짐' if isinstance(x, str) and x == '넘어짐(기타)' else x)

test_df['공사종류(대분류)'] = test_df['공사종류'].str.split(' / ').str[0]
test_df['공사종류(중분류)'] = test_df['공사종류'].str.split(' / ').str[1]
test_df['공종(대분류)'] = test_df['공종'].str.split(' > ').str[0]
test_df['공종(중분류)'] = test_df['공종'].str.split(' > ').str[1]
test_df['사고객체(대분류)'] = test_df['사고객체'].str.split(' > ').str[0]
test_df['사고객체(중분류)'] = test_df['사고객체'].str.split(' > ').str[1]
test_df['인적사고'] = test_df['인적사고'].apply(lambda x: '떨어짐' if '떨어짐' in x else x)
test_df['인적사고'] = test_df['인적사고'].apply(lambda x: '넘어짐' if x == '넘어짐(기타)' else x)

In [55]:
def category_template(row):
    text = (
        f"{row['작업프로세스']} 중 " 
        if (row['작업프로세스'] != '기타' and not pd.isna(row['작업프로세스']))
        else ""
    ) + (
        f"{row['사고객체(중분류)']} 관련 " 
        if (row['사고객체(중분류)'] != '기타' and not pd.isna(row['사고객체(중분류)']) and not (row['사고객체(중분류)'] == '질병' and row['인적사고'] == '질병'))
        else ""
    ) + (
        f"{row['인적사고']} " 
        if (row['인적사고'] not in ['기타', '분류불능', '없음'] and not pd.isna(row['인적사고']))
        else ""
    ) + (
        f"및 " 
        if ((row['인적사고'] not in ['기타', '분류불능', '없음'] and not pd.isna(row['인적사고'])) and 
            (row['물적사고'] not in ['기타', '없음'] and not pd.isna(row['물적사고'])))
        else ""
    ) + (
        f"{row['물적사고']} " 
        if (row['물적사고'] not in ['기타', '없음'] and not pd.isna(row['물적사고']))
        else ""
    )

    return text

In [56]:
train_df['category'] = train_df.apply(lambda x: category_template(x), axis=1)
test_df['category'] = test_df.apply(lambda x: category_template(x), axis=1)

train_df['category_exists'] = ""
test_df['category_exists'] = ""

train_df.loc[train_df['category']!="", "category_exists"] = "Y"
test_df.loc[test_df['category']!="", "category_exists"] = "Y"

train_df.loc[train_df['category']=="", "category_exists"] = "N"
test_df.loc[test_df['category']=="", "category_exists"] = "N"

In [57]:
def description_template(row):
    if row['category_exists'] == "Y":
        text = row['category'] + f"사고 ({row['사고원인']})"
    else:
        text = row['사고원인']
    return text

# 훈련 데이터 통합 생성
combined_training_data = train_df.apply(
    lambda row: {
        "description": description_template(row),
        "ground_truth": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [58]:
# 테스트 데이터 통합 생성
combined_test_data = test_df.apply(
    lambda row: {
        "description": description_template(row)
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

In [59]:
train_df = train_df.assign(description=combined_training_data['description'])
train_df = train_df.assign(ground_truth=combined_training_data['ground_truth'])

test_df = test_df.assign(description=combined_test_data['description'])

In [60]:
# NaN값 확인
print("Train Description의 NaN 개수:", train_df['description'].isna().sum())
print("Train Answer의 NaN 개수:", train_df['ground_truth'].isna().sum())

print("Test Description의 NaN 개수:", test_df['description'].isna().sum())


Train Description의 NaN 개수: 0
Train Answer의 NaN 개수: 0
Test Description의 NaN 개수: 0


In [61]:
train_df = train_df[['category_exists', 'description', 'ground_truth']]
test_df = test_df[['category_exists', 'description']]

In [62]:
train_df.to_csv('../data/train_01.csv', index_label='id')
test_df.to_csv('../data/test_01.csv', index_label='id')